In [ ]:
# 1. Run `source setpath.sh` first!
# 2. Remember to enable `packet_rgb_double` in the config
import os
import mitsuba
mitsuba.set_variant('packet_rgb_double')

In [ ]:
from mitsuba.core import FileStream
from mitsuba.render import Integrator
from mitsuba.core import Thread
from mitsuba.core import Bitmap
from mitsuba.core import Struct
from mitsuba.core import LogLevel
from mitsuba.core import xml
from mitsuba.core.xml import load_file

In [ ]:
def render_to_jpg(src_file, dst_file, sensor_idx=0):
    src_file, dst_file = str(src_file), str(dst_file)
    if not src_file.endswith(".xml"):
        raise ValueError("src_file must be .xml")
    if not dst_file.endswith(".jpg") and not dst_file.endswith(".png"):
        raise ValueError("dst_file must be .jpg")
    
    # Load the scene for an XML file
    Thread.thread().logger().set_error_level(LogLevel.Error)
    Thread.thread().file_resolver().append(os.path.dirname(src_file))
    scene = load_file(src_file)
    
    # Get the scene's sensor (if many, can pick one by specifying the index)
    num_sensors = len(scene.sensors())
    if sensor_idx >= num_sensors:
        raise ValueError(f"num_sensors={num_sensors}, sensor_idx={sensor_idx} is invalid")
    else:
        print(f"num_sensors={num_sensors}, selected={sensor_idx}")
    sensor = scene.sensors()[sensor_idx]
    
    # Call the scene's integrator to render the loaded scene with the desired sensor
    scene.integrator().render(scene, sensor)
    
    # The rendered data is stored in the film
    film = sensor.film()
    img = film.bitmap(raw=True).convert(Bitmap.PixelFormat.RGB, Struct.Type.UInt8, srgb_gamma=True)
    img.write(dst_file)
    print(f"Image written to {dst_file}")

In [ ]:
src_file = "resources/data/scenes/cbox/cbox-rgb.xml"
dst_file = "resources/data/scenes/cbox/cbox-rgb.png"
render_to_jpg(src_file, dst_file)